In [3]:
import IPython.display as ipd
import numpy as np

# Part 1: Karplus-Strong

In [4]:
def karplus_strong_note(sr, note, duration, decay):
    """
    Parameters
    ----------
    sr: int
        Sample rate
    note: int
        Note number.  0 is 440hz concert A
    duration: float
        Seconds of audio
    decay: float 
        Decay amount (between 0 and 1)

    Returns
    -------
    ndarray(N): Audio samples for this note
    """
    N = int(duration*sr)
    y = np.zeros(N)
    ## TODO: Fill this in
    T = int(sr/(440*2**(note/12)))
    y[:T] = np.random.randn(T)
    for i in range(T, N):
        y[i] = decay * (y[i - T] + y[i - T + 1])/2
    return y


sr = 44100
note = -12
duration = 1
decay = 0.98
x = karplus_strong_note(sr, note, duration, decay)
#plt.plot(x)
#plt.show()
ipd.Audio(x, rate=sr)

# Part 2: Tune Making

In [5]:
def make_tune(filename, sixteenth_len, sr, note_fn):
    """
    Parameters
    ----------
    filename: string
        Path to file containing the tune.  Consists of
        rows of <note number> <note duration>, where
        the note number 0 is a 440hz concert A, and the
        note duration is in factors of 16th notes
    sixteenth_len: float
        Length of a sixteenth note, in seconds
    sr: int
        Sample rate
    note_fn: function (sr, note, duration) -> ndarray(M)
        A function that generates audio samples for a particular
        note at a given sample rate and duration
    
    Returns
    -------
    ndarray(N): Audio containing the tune
    """
    tune = np.loadtxt(filename)
    notes = tune[:, 0]
    durations = sixteenth_len*tune[:, 1]
    ## TODO: Fill this in
    y = np.array([])
    for note, duration in zip(notes, durations):
        y = np.concatenate((y, note_fn(sr, note, duration)))
    return y

In [8]:
sr = 44100
note_fn = lambda sr, note, duration: karplus_strong_note(sr, note, duration, 0.99)
y = make_tune("Tunes/missy.txt", 0.18, 44100, note_fn)
ipd.Audio(y, rate=sr)

In [7]:
sr = 44100
note_fn = lambda sr, note, duration: karplus_strong_note(sr, note-12, duration, 0.92)
y = make_tune("Tunes/missy.txt", 0.18, 44100, note_fn)
ipd.Audio(y, rate=sr)

# Part 3: FM Synthesis Framework

In [30]:
from instruments import *

def fm_synth_note(sr, note, duration, ratio=2, I=2, 
                  envelope = lambda N, sr: np.ones(N),
                  amplitude = lambda N, sr: np.ones(N)):
    """
    Parameters
    ----------
    sr: int
        Sample rate
    note: int
        Note number.  0 is 440hz concert A
    duration: float
        Seconds of audio
    ratio: float
        Ratio of modulation frequency to carrier frequency
    I: float
        Modulation index (ratio of peak frequency deviation to
        modulation frequency)
    envelope: function (N, sr) -> ndarray(N)
        A function for generating an ADSR profile
    amplitude: function (N, sr) -> ndarray(N)
        A function for generating a time-varying amplitude

    Returns
    -------
    ndarray(N): Audio samples for this note
    """
    N = int(duration*sr)
    y = np.zeros(N)
    ## TODO: Fill this in
    fc = 440*2**(note/12)
    fm = ratio*fc
    t = np.arange(N)/sr
    y = amplitude(N, sr)*np.cos(2*np.pi*fc*t + (envelope(N,sr)*I)*np.sin(2*np.pi*fm*t))
    
    return y

In [31]:
sr = 44100
note_fn = lambda sr, note, duration: fm_plucked_string_note(sr, note, duration, 8)
y = make_tune("Tunes/dangerzone.txt", 0.1, 44100, note_fn)
ipd.Audio(y, rate=sr)

In [32]:
sr = 44100
note_fn = lambda sr, note, duration: fm_plucked_string_note(sr, note-12, duration, 8)
y = make_tune("Tunes/dangerzone.txt", 0.1, 44100, note_fn)
ipd.Audio(y, rate=sr)